# Laboratorio 3 - Data Science
Diego Morales - 21146   
Alejandro Ortega - 18248

[Enlace al Repositorio de Github](https://github.com/Diego2250/Lab-3-DS.git)

In [78]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import pickle
import os
from tensorflow.keras.preprocessing.image import img_to_array
import tensorflow as tf
from tensorflow.keras import layers, models
import random


In [79]:
# Establecer la semilla para reproducibilidad
seed = 42
np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

In [80]:
print(f"Tensor flow version: {tf.__version__}")

Tensor flow version: 2.17.0


## 1. Preparación de datos

In [81]:
# Cargar el archivo .p
with open('Datos_Rotulos_Trafico/entrenamiento.p', 'rb') as file:
    data = pickle.load(file)

# Imprimir las claves disponibles
print(data.keys())

# Inspeccionar un poco más el contenido
for key in data.keys():
    print(f"Key: {key}, Type: {type(data[key])}, Length: {len(data[key])}")

dict_keys(['coords', 'labels', 'features', 'sizes'])
Key: coords, Type: <class 'numpy.ndarray'>, Length: 34799
Key: labels, Type: <class 'numpy.ndarray'>, Length: 34799
Key: features, Type: <class 'numpy.ndarray'>, Length: 34799
Key: sizes, Type: <class 'numpy.ndarray'>, Length: 34799


In [82]:
def load_data(pickle_file):
    with open(pickle_file, 'rb') as file:
        data = pickle.load(file)
    return data['features'], data['labels']

In [83]:
X_train, y_train = load_data('Datos_Rotulos_Trafico/entrenamiento.p')
X_validation, y_validation = load_data('Datos_Rotulos_Trafico/validacion.p')
X_test, y_test = load_data('Datos_Rotulos_Trafico/prueba.p')

In [84]:
X_train = X_train / 255.0
X_validation = X_validation / 255.0
X_test = X_test / 255.0


y_train = np.array(y_train)
y_validation = np.array(y_validation)
y_test = np.array(y_test)

In [85]:
X_train.shape

(34799, 32, 32, 3)

In [86]:
y_train.shape

(34799,)

In [87]:
X_validation.shape

(4410, 32, 32, 3)

In [88]:
y_validation.shape

(4410,)

In [89]:
X_test.shape

(12630, 32, 32, 3)

In [90]:
y_test.shape

(12630,)

## 2. Implementación de la arquitectura Le-Net

### Presentar la arquitectura Le-Net en detalle, explicando cada capa (convolucional, pooling, fully connected)

#### Capas convolucionales

Son responsables de detectar patrones en las imágenes de entrada, como bordes, texturas, o formas específicas mediante la aplicación de filtros convolucionales. Cada filtro se "desplaza" sobre la imagen (operación de convolución) para generar un mapa de características cpn el fin de resaltar la presencia de dichos patrones en diferentes regiones de la imagen. La arquitectura Le-Net cuenta con dos capas convolucionales principales, una para detectar las características generales y otra para detectar las características más complejas. 

#### Capas de pooling (submuestreo)

Estas capas tienen el propósito de reducir la dimensionalidad espacial de los mapas de características y se colocan después de las capas convolucionales, lo cual disminuye la cantidad de parámetros y la complejidad computacional, ayudando a prevenir el sobreajuste. En Le-Net, se suele utilizar *average pooling*, el cual toma el promedio de una región de la imagen para reducir su tamaño, preservando la información más significativa.

#### Capas fully connected

Estas capas están situadas al final de la red y conectan todas las neuronas de la capa previa con todas las neuronas de la capa actual. Su función es combinar las características extraídas por las capas anteriores para obtener la clasificación final. En Le-Net, estas capas conducen a una capa de salida que tiene tantas neuronas como clases de salida (en este caso, 43 neuronas correspondientes a las 43 clases de rótulos de tráfico).

### Mostrar el diseño de la red Le-Net utilizando una herramienta de diagramación

![Diagrama de Arquitectura](./img/Diagrama_Arquitectura_Lab03_light.png)

### Explicar el proceso de convolución, función de activación y pooling.

#### Proceso de convolución

La convolución es una operación matemática donde un filtro (un pequeño conjunto de pesos) se aplica sobre la imagen de entrada para detectar características locales. A medida que el filtro se mueve por la imagen, calcula el producto punto entre el filtro y la porción de la imagen que cubre, generando un valor de salida en el mapa de características.

#### Función de activación

**ReLU (Rectified Linear Unit)** es una función de activación comúnmente utilizada, que introduce no linealidad en la red, permitiendo que ésta aprenda representaciones complejas. ReLU convierte los valores negativos en cero y deja pasar los valores positivos sin cambio.<br/>
<br/>
$$f(x)=max(0,1)$$

#### Pooling/Submuestreo

**MaxPooling** es una operación que se aplica después de las capas convolucionales para reducir las dimensiones espaciales (ancho y alto) de los mapas de características, manteniendo la información más relevante. Esta reducción se realiza mediante la selección del valor máximo en cada ventana de la operación de pooling.

## 3. Contrucción de modelo

In [104]:
model = models.Sequential()

In [105]:
# Definir la estructura de capas convolucionales, capas de pooling y capas fully connected
model.add(layers.Input(shape=(32, 32, 3)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

model.add(layers.Flatten())

model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(43, activation='softmax'))


In [93]:
model.summary()

Model: "sequential_5"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_14 (Conv2D)              │ (None, 30, 30, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 15, 15, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_15 (Conv2D)              │ (None, 13, 13, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 6, 6, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_16 (Conv2D)              │ (None, 4, 4, 64)       │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_4 (Flatten)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 64)             │        65,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 43)             │         2,795 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 124,715 (487.17 KB)

 Trainable params: 124,715 (487.17 KB)

 Non-trainable params: 0 (0.00 B)

In [94]:
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

### Explicar la importancia de la función de pérdida y el optimizador.

## 4. Entrenamiento del modelo

### Explicar el proceso de entrenamiento de la red neuronal.


In [95]:
# Mostrar cómo cargar los datos de entrenamiento y validación en lotes.
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_dataset = train_dataset.shuffle(len(X_train)).batch(128)

validation_dataset = tf.data.Dataset.from_tensor_slices((X_validation, y_validation))
validation_dataset = validation_dataset.batch(128)

# Compilar el modelo
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [96]:
history = model.fit(train_dataset, epochs=5, validation_data=validation_dataset)

Epoch 1/5
272/272 ━━━━━━━━━━━━━━━━━━━━ 15s 47ms/step - accuracy: 0.3065 - loss: 2.6534 - val_accuracy: 0.7773 - val_loss: 0.8047
Epoch 2/5
272/272 ━━━━━━━━━━━━━━━━━━━━ 13s 46ms/step - accuracy: 0.8895 - loss: 0.4109 - val_accuracy: 0.8644 - val_loss: 0.5405
Epoch 3/5
272/272 ━━━━━━━━━━━━━━━━━━━━ 12s 44ms/step - accuracy: 0.9554 - loss: 0.1810 - val_accuracy: 0.8726 - val_loss: 0.4350
Epoch 4/5
272/272 ━━━━━━━━━━━━━━━━━━━━ 12s 44ms/step - accuracy: 0.9736 - loss: 0.1088 - val_accuracy: 0.9098 - val_loss: 0.3790
Epoch 5/5
272/272 ━━━━━━━━━━━━━━━━━━━━ 12s 43ms/step - accuracy: 0.9816 - loss: 0.0712 - val_accuracy: 0.9175 - val_loss: 0.3745


In [97]:
print(history.history.keys())

dict_keys(['accuracy', 'loss', 'val_accuracy', 'val_loss'])


In [98]:
print(f"Accuracy: {history.history['accuracy']}")

Accuracy: [0.538981020450592, 0.9150550365447998, 0.9608609676361084, 0.975372850894928, 0.9823271036148071]


## 5. Evaluación del modelo

In [99]:
# Evaluar el modelo entrenado utilizando el conjunto de prueba.
test_loss, test_accuracy = model.evaluate(X_test, y_test)

395/395 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.9082 - loss: 0.4870


In [100]:
# Mostrar cómo calcular métricas de evaluación, como Precisión, Recall y F1-Score para cada clase
y_pred = model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)

from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

395/395 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
              precision    recall  f1-score   support

           0       0.94      0.48      0.64        60
           1       0.84      0.98      0.91       720
           2       0.95      0.94      0.95       750
           3       0.92      0.93      0.93       450
           4       0.96      0.91      0.94       660
           5       0.88      0.87      0.88       630
           6       0.99      0.71      0.83       150
           7       0.94      0.80      0.87       450
           8       0.83      0.94      0.88       450
           9       0.94      0.99      0.96       480
          10       0.99      0.95      0.97       660
          11       0.92      0.90      0.91       420
          12       0.93      0.95      0.94       690
          13       0.99      0.96      0.98       720
          14       0.99      0.99      0.99       270
          15       0.97      0.95      0.96       210
          16       1.00      0.97      0

### Interpretación de resultados
La red Le-Net muestra un rendimiento general bastante bueno con una precisión del 90% en el conjunto de prueba. Las métricas de evaluación indican que el modelo tiene un alto desempeño en la mayoría de las clases, donde la precisión y el recall son elevados. Sin embargo, en ciertas clases, el modelo presenta una precisión más baja o un recall insuficiente, lo que sugiere que podría ser útil ajustar el modelo, aumentar la cantidad de datos de entrenamiento o aplicar técnicas de regularización para mejorar el rendimiento en estas clases.